In [3]:
file_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\office-projects\pdf-extractor\data\scanned-pdf\fact_sheet_all_data.pdf"

In [23]:
import pdfplumber

with pdfplumber.open(file_path) as pdf:
    first_page = pdf.pages[0]
    print(first_page.chars[0])
    print(f"The class of pdf is {type(first_page)}", first_page.width, first_page.height)
    #print(f"Lines: {first_page.lines}")
    #print(f"Rects: {first_page.rects}")

    tables = first_page.extract_tables()
    print(len(tables))

{'matrix': (9, 0, 0, 9, 587.8828, 20.7949), 'fontname': 'VWUJSI+OpenSans', 'adv': 0.5720000000000001, 'upright': True, 'x0': 587.8828, 'y0': 18.3559, 'x1': 593.0308, 'y1': 27.3559, 'width': 5.148000000000025, 'height': 9.0, 'size': 9.0, 'mcid': None, 'tag': 'Layer', 'object_type': 'char', 'page_number': 1, 'ncs': 'ICCBased', 'text': '6', 'stroking_color': None, 'stroking_pattern': None, 'non_stroking_color': (0.137, 0.123, 0.126), 'non_stroking_pattern': None, 'top': 764.6441, 'bottom': 773.6441, 'doctop': 764.6441}
The class of pdf is <class 'pdfplumber.page.Page'> 612 792
3


In [ ]:
def extract_page_tables(page):

    tables = page.extract_tables()
    for table in tables:
        print(table)

def get_page_data(file_path):

    with pdfplumber.open(file_path) as pdf:

        pages = list(_ for _ in pdf.pages)
        print(pages, type(pages[0]))

        for page in pages:
            extract_page_tables(page)




get_page_data(file_path)

[<Page:1>] <class 'pdfplumber.page.Page'>
[['Index Name', 'As on', None, None, 'Return in %', None], [None, '29-Nov-24', '31-Oct-24', '30-Nov-23', '1 Month', '1 Year'], ['Nifty 50\nS&P BSE Sensex\nS&P BSE MID CAP\nS&P BSE SMALL CAP\nS&P BSE 200\nS&P BSE AUTO\nS&P BSE Bankex\nS&P BSE Consumer Durable\nS&P BSE Capital Good\nS&P BSE FMCG\nS&P BSE Health Care\nS&P BSE IT\nS&P BSE METAL\nS&P BSE Oil & Gas\nS&P BSE Power Index\nS&P BSE Realty', '24131\n79803\n46071\n55200\n11123\n52898\n59298\n62469\n70700\n21213\n43666\n42783\n30537\n26813\n7490\n7960', '24205\n79389\n45967\n54983\n11121\n53540\n58664\n60656\n69106\n21663\n43915\n40428\n31280\n27458\n7829\n7809', '20133\n66988\n34256\n40372\n8905\n40053\n50293\n47120\n49990\n19157\n30375\n33227\n24240\n20551\n4921\n5657', '-0.3\n0.5\n0.2\n0.4\n0.0\n-1.2\n1.1\n3.0\n2.3\n-2.1\n-0.6\n5.8\n-2.4\n-2.3\n-4.3\n1.9', '19.9\n19.1\n34.5\n36.7\n24.9\n32.1\n17.9\n32.6\n41.4\n10.7\n43.8\n28.8\n26.0\n30.5\n52.2\n40.7']]
[['Index Name', 'As on', None, Non